In [128]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [129]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-d7d4df0e-05b7-4541-b09c-4daacb8dda9f'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [130]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'Kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [172]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match("EUROPE", matchId)
    if match_detail["info"]["mapId"] in [1, 2, 11]:
        match = {}
        match["matchId"] = matchId
        match["gameDuration"] = match_detail["info"]["gameDuration"]
        match["gameType"] = match_detail["info"]["gameType"]
        match["gameVersion"] = match_detail["info"]["gameVersion"]
        matches.append(match)
        for index, row in enumerate(match_detail["info"]["participants"]):
            participants_row = {}
            participants_row["matchId"] = matchId
            participants_row["participant"] = index + 1
            participants_row["summonerName"] = row["summonerName"]
            participants_row["teamPosition"] = row["teamPosition"]
            participants_row["championName"] = row["championName"]
            participants_row["kda"] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
            participants_row["champLevel"] = row["champLevel"]
            participants_row["totalDamageToChampions"] = row[
                "totalDamageDealtToChampions"
            ]
            participants_row["totalDamageTaken"] = row["totalDamageTaken"]
            participants_row["totalHeal"] = row["totalHeal"]
            participants_row["totalMinionsKilled"] = row["totalMinionsKilled"]
            participants_row["goldEarned"] = row["goldEarned"]
            participants_row["summoner1"] = row["summoner1Id"]
            participants_row["summoner2"] = row["summoner2Id"]
            participants_row["item0"] = row["item0"]
            participants_row["item1"] = row["item1"]
            participants_row["item2"] = row["item2"]
            participants_row["item3"] = row["item3"]
            participants_row["item4"] = row["item4"]
            participants_row["item5"] = row["item5"]
            participants_row["item6"] = row["item6"]
            participants_row["win"] = row["win"]
            participants.append(participants_row)

        for row in timeline["info"]["frames"]:
            # frameCounter = 1
            # frame = row["{:02d}".format(frameCounter)]
            # frameCounter += 1
            for frameRow in row["events"]:
                timeline_row = {}
                timeline_row["matchId"] = matchId
                eventCounter = 0
                # eventFrame = frameRow["{:02d}".format(eventCounter)]
                eventFrame = frameRow
                eventCounter += 1

                if eventFrame["type"] == "CHAMPION_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["victimId"] = eventFrame["victimId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    if "bounty" in eventFrame:
                        timeline_row["bounty"] = eventFrame["bounty"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "BUILDING_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["teamId"]
                    timeline_row["buildingType"] = eventFrame["buildingType"]
                    timeline_row["laneType"] = eventFrame["laneType"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ITEM_PURCHASED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["itemId"] = eventFrame["itemId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_PLACED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["creatorId"] = eventFrame["creatorId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "LEVEL_UP":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["level"] = eventFrame["level"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ELITE_MONSTER_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["killerTeamId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    timeline_row["monsterType"] = eventFrame["monsterType"]
                    matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
matchTimelineDf["timestampMIN"] = round(matchTimelineDf["timestamp"] / 60000, 0)


In [132]:
match_detail["info"]["mapId"]

11

In [133]:
participantsDf

,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5808285447,1,Kreatonn,TOP,Sion,0/7/6,16,7133,27598,644,...,4,12,1011,3181,0,3068,3047,3077,3340,True
1,EUW1_5808285447,2,Forcanist,JUNGLE,Darius,8/8/8,16,23119,42132,15433,...,6,11,3742,6631,4401,3111,3133,1031,3364,True
2,EUW1_5808285447,3,Hisshiden,MIDDLE,Ahri,24/5/5,18,35388,22712,9141,...,4,14,3020,6656,3157,3041,3135,3089,3364,True
3,EUW1_5808285447,4,Safiphy,BOTTOM,Aphelios,14/5/9,15,25949,17747,3910,...,3,4,2031,6671,6676,3031,3036,3006,3363,True
4,EUW1_5808285447,5,TarKy,UTILITY,Janna,3/4/24,15,7622,16187,15556,...,14,4,3853,2065,3107,3504,3158,3801,3364,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,EUW1_5747855273,6,Junipie,TOP,Illaoi,3/3/1,15,9241,10082,1507,...,4,12,6632,3047,3133,3181,1031,1037,3340,False
126,EUW1_5747855273,7,NightWõlf,JUNGLE,Viego,2/4/3,12,8777,19859,6826,...,4,11,3047,2031,6632,1053,1037,1042,3340,False
127,EUW1_5747855273,8,Kelody,MIDDLE,Ahri,0/9/1,10,8515,13651,1479,...,4,14,1056,6655,3020,1052,0,0,3340,False
128,EUW1_5747855273,9,Sir Nurum,BOTTOM,Xayah,2/9/1,10,8737,14138,865,...,7,4,1018,0,6671,3006,1037,3134,3340,False


In [134]:
matchTimelineDf["assistingParticipantIds"] = [ [] if x is np.NaN else x for x in matchTimelineDf["assistingParticipantIds"]]

In [135]:
def getSummonerName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = int(row["killerId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = int(row["victimId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def getChampionName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = row["killerId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = row["victimId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def checkAssist(row, name):
    matchId = row["matchId"]
    if row["assistingParticipantIds"]:
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0] == name:
                return True
    return False


In [136]:
# matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
# matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
# matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)

0       False
1       False
2       False
3       False
4       False
        ...  
8308    False
8309    False
8310    False
8311    False
8312    False
Length: 8313, dtype: bool

In [137]:
matchTimelineDf["participantName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
matchTimelineDf["participantChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "participant"), axis=1)
matchTimelineDf["killerName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
matchTimelineDf["killerChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "killer"), axis=1)
matchTimelineDf["victimName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf["victimChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "victim"), axis=1)
matchTimelineDf["kelodyAssist"] = matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)


In [138]:
matchTimelineDf

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5808285447,ITEM_PURCHASED,6.0,2033.0,2750,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,five night fredy,Gangplank,NaN,NaN,NaN,NaN,False
1,EUW1_5808285447,ITEM_PURCHASED,6.0,3340.0,3278,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,five night fredy,Gangplank,NaN,NaN,NaN,NaN,False
2,EUW1_5808285447,ITEM_PURCHASED,7.0,1035.0,6217,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
3,EUW1_5808285447,ITEM_PURCHASED,7.0,3340.0,6613,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
4,EUW1_5808285447,ITEM_PURCHASED,7.0,2031.0,6911,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8308,EUW1_5747855273,WARD_KILL,NaN,NaN,1321811,NaN,UNDEFINED,NaN,4.0,NaN,...,NaN,NaN,22.0,NaN,NaN,ROHNEN1,Caitlyn,NaN,NaN,False
8309,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1323242,NaN,NaN,NaN,5.0,9.0,...,NaN,NaN,22.0,NaN,NaN,KG Maisen,Pyke,Sir Nurum,Xayah,False
8310,EUW1_5747855273,LEVEL_UP,5.0,NaN,1323242,12.0,NaN,NaN,NaN,NaN,...,NaN,NaN,22.0,KG Maisen,Pyke,NaN,NaN,NaN,NaN,False
8311,EUW1_5747855273,WARD_PLACED,NaN,NaN,1323676,NaN,UNDEFINED,6.0,NaN,NaN,...,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,False


In [139]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
6,EUW1_5808285447,7
11,EUW1_5808262634,2
26,EUW1_5803852070,7
36,EUW1_5803774910,7
41,EUW1_5803660385,2
51,EUW1_5803299160,2
61,EUW1_5803373057,2
71,EUW1_5803198068,2
86,EUW1_5803065394,7
96,EUW1_5802442397,7


In [140]:
kelodyEvents = []
#TODO bool if kelody was assist or not
# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5808285447,ITEM_PURCHASED,7.0,1035.0,6217,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
1,EUW1_5808285447,ITEM_PURCHASED,7.0,3340.0,6613,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
2,EUW1_5808285447,ITEM_PURCHASED,7.0,2031.0,6911,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
3,EUW1_5808285447,LEVEL_UP,7.0,NaN,106391,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
4,EUW1_5808285447,LEVEL_UP,7.0,NaN,141413,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,EUW1_5747855273,LEVEL_UP,8.0,NaN,1154602,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,19.0,Kelody,Ahri,NaN,NaN,NaN,NaN,False
1093,EUW1_5747855273,ITEM_PURCHASED,8.0,6655.0,1168231,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,19.0,Kelody,Ahri,NaN,NaN,NaN,NaN,False
1094,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1256497,NaN,NaN,NaN,2.0,8.0,...,NaN,NaN,21.0,NaN,NaN,Takeasy,Evelynn,Kelody,Ahri,False
1095,EUW1_5747855273,ITEM_PURCHASED,8.0,1052.0,1282981,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,21.0,Kelody,Ahri,NaN,NaN,NaN,NaN,False


In [141]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == True]

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
6,EUW1_5808285447,CHAMPION_KILL,NaN,NaN,223568,NaN,NaN,NaN,6.0,2.0,...,NaN,NaN,4.0,NaN,NaN,five night fredy,Gangplank,Forcanist,Darius,True
7,EUW1_5808285447,CHAMPION_KILL,NaN,NaN,231530,NaN,NaN,NaN,6.0,1.0,...,NaN,NaN,4.0,NaN,NaN,five night fredy,Gangplank,Kreatonn,Sion,True
15,EUW1_5808285447,CHAMPION_KILL,NaN,NaN,511660,NaN,NaN,NaN,9.0,5.0,...,NaN,NaN,9.0,NaN,NaN,Mini Wolf,Jinx,TarKy,Janna,True
23,EUW1_5808285447,CHAMPION_KILL,NaN,NaN,687428,NaN,NaN,NaN,6.0,3.0,...,NaN,NaN,11.0,NaN,NaN,five night fredy,Gangplank,Hisshiden,Ahri,True
38,EUW1_5808285447,CHAMPION_KILL,NaN,NaN,969957,NaN,NaN,NaN,9.0,2.0,...,NaN,NaN,16.0,NaN,NaN,Mini Wolf,Jinx,Forcanist,Darius,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,919417,NaN,NaN,NaN,7.0,4.0,...,NaN,NaN,15.0,NaN,NaN,NightWõlf,Karthus,GranpaLongShlong,MissFortune,True
1018,EUW1_5747889998,ELITE_MONSTER_KILL,NaN,NaN,934511,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,16.0,NaN,NaN,NightWõlf,Karthus,NaN,NaN,True
1019,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,974344,NaN,NaN,NaN,7.0,1.0,...,NaN,NaN,16.0,NaN,NaN,NightWõlf,Karthus,TrouserLeviathan,MonkeyKing,True
1035,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,1228414,NaN,NaN,NaN,10.0,4.0,...,NaN,NaN,20.0,NaN,NaN,Fegel,Braum,GranpaLongShlong,MissFortune,True


Check how often a Player died in Early Game (pre 10 mins)

In [142]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
6,EUW1_5808285447,7,1
11,EUW1_5808262634,2,2
26,EUW1_5803852070,7,1
36,EUW1_5803774910,7,1
41,EUW1_5803660385,2,2
51,EUW1_5803299160,2,1
61,EUW1_5803373057,2,3
71,EUW1_5803198068,2,1
86,EUW1_5803065394,7,1
96,EUW1_5802442397,7,3


#### TODO
##### Features:
- <s>Died from ganks in laning phase pre 15 min (check ID from enemy Jungler)</s>

- Died with ... Gold

- <s>count 1v1 kill/deaths</s>  
  
- Good roaming (assists pre 15 min)

- Warding (check ward placements pre 15 min) maybe in combination with ganks

- CS per min (if data available)

- KDA (kills and assists divided by deaths) als Output KDA String aus participantDf

- <s>Tower Kills/Assists</s> 

- <s>Objective Kills/Assists (Baron, Dragon, Herald)</s> 



##### Features on the tableau visualization:

- <s>Timeline</s>

- <s>Champion Icons</s>

- <s>Damage Dealt Bar Chart (participantDf)</s>

- Game win (participantDf)



Check 1v1

In [143]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == False]

def check1v1(row, name):
    if not row["assistingParticipantIds"] and (row["victimName"] == name or row["killerName"] == name) and (row["type"] == "CHAMPION_KILL"):
        return True
    return False

kelodyEventsDf["1v1"] = kelodyEventsDf.apply(lambda row: check1v1(row, "Kelody"), axis = 1)
kelodyEventsDf[kelodyEventsDf["1v1"] == True]

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist,1v1
51,EUW1_5808285447,CHAMPION_KILL,NaN,NaN,1549618,NaN,NaN,NaN,3.0,7.0,...,NaN,26.0,NaN,NaN,Hisshiden,Ahri,Kelody,Diana,False,True
69,EUW1_5808262634,CHAMPION_KILL,NaN,NaN,191543,NaN,NaN,NaN,7.0,2.0,...,NaN,3.0,NaN,NaN,Davidodos,Evelynn,Kelody,Talon,False,True
112,EUW1_5808262634,CHAMPION_KILL,NaN,NaN,1080929,NaN,NaN,NaN,8.0,2.0,...,NaN,18.0,NaN,NaN,Kegelrobbe,Akali,Kelody,Talon,False,True
118,EUW1_5808262634,CHAMPION_KILL,NaN,NaN,1146939,NaN,NaN,NaN,2.0,9.0,...,NaN,19.0,NaN,NaN,Kelody,Talon,Käptn Bubátz,Caitlyn,False,True
122,EUW1_5808262634,CHAMPION_KILL,NaN,NaN,1194905,NaN,NaN,NaN,2.0,8.0,...,NaN,20.0,NaN,NaN,Kelody,Talon,Kegelrobbe,Akali,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,512048,NaN,NaN,NaN,3.0,8.0,...,NaN,9.0,NaN,NaN,r3dcactus,Neeko,Kelody,Ahri,False,True
1080,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,652234,NaN,NaN,NaN,3.0,8.0,...,NaN,11.0,NaN,NaN,r3dcactus,Neeko,Kelody,Ahri,False,True
1088,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1017707,NaN,NaN,NaN,5.0,8.0,...,NaN,17.0,NaN,NaN,KG Maisen,Pyke,Kelody,Ahri,False,True
1094,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1256497,NaN,NaN,NaN,2.0,8.0,...,NaN,21.0,NaN,NaN,Takeasy,Evelynn,Kelody,Ahri,False,True


In [144]:
def count1v1(row, name, killer):
  if killer:
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["killerName"] == name)])
  else: 
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["victimName"] == name)])

kelody["1v1Won"] = kelody.apply(lambda row: count1v1(row, "Kelody", True), axis=1)
kelody["1v1Lost"] = kelody.apply(lambda row: count1v1(row, "Kelody", False), axis=1)
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost
6,EUW1_5808285447,7,1,0,1
11,EUW1_5808262634,2,2,2,2
26,EUW1_5803852070,7,1,11,0
36,EUW1_5803774910,7,1,7,1
41,EUW1_5803660385,2,2,5,1
51,EUW1_5803299160,2,1,2,0
61,EUW1_5803373057,2,3,1,1
71,EUW1_5803198068,2,1,3,1
86,EUW1_5803065394,7,1,3,5
96,EUW1_5802442397,7,3,7,2


In [145]:
def countTurretKills(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "BUILDING_KILL") & (kelodyEventsDf["killerName"] == name) & (kelodyEventsDf["1v1"] == False)])

kelody["turretKills"] = kelody.apply(lambda row: countTurretKills(row, "Kelody"), axis=1)
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills
6,EUW1_5808285447,7,1,0,1,0
11,EUW1_5808262634,2,2,2,2,1
26,EUW1_5803852070,7,1,11,0,1
36,EUW1_5803774910,7,1,7,1,0
41,EUW1_5803660385,2,2,5,1,0
51,EUW1_5803299160,2,1,2,0,6
61,EUW1_5803373057,2,3,1,1,1
71,EUW1_5803198068,2,1,3,1,1
86,EUW1_5803065394,7,1,3,5,2
96,EUW1_5802442397,7,3,7,2,1


In [146]:
def counEliteMonsterKills(row, monsterName):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "ELITE_MONSTER_KILL") & (kelodyEventsDf["monsterType"] == monsterName)])

kelody["dragonKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "DRAGON"), axis=1)
kelody["baronKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "BARON_NASHOR"), axis=1)
kelody["heraldKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "RIFTHERALD"), axis=1)

kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills
6,EUW1_5808285447,7,1,0,1,0,0,0,0
11,EUW1_5808262634,2,2,2,2,1,2,0,1
26,EUW1_5803852070,7,1,11,0,1,3,0,1
36,EUW1_5803774910,7,1,7,1,0,2,0,1
41,EUW1_5803660385,2,2,5,1,0,3,0,0
51,EUW1_5803299160,2,1,2,0,6,1,0,2
61,EUW1_5803373057,2,3,1,1,1,2,1,1
71,EUW1_5803198068,2,1,3,1,1,3,0,0
86,EUW1_5803065394,7,1,3,5,2,4,1,1
96,EUW1_5802442397,7,3,7,2,1,1,0,1


In [147]:
def deathToGank(row, name):
    if row["1v1"] == False and row["type"] == "CHAMPION_KILL" and row["victimName"] == name: 
        enemyJungler = participantsDf.loc[(row["matchId"] == participantsDf["matchId"]) & (participantsDf["teamPosition"] == "JUNGLE")]["summonerName"].values[0]
        if row["killerName"] == enemyJungler:
            return True
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == row["matchId"])]["teamPosition"].values[0] == "JUNGLE":
                return True
    return False
kelodyEventsDf["deathToGank"] = kelodyEventsDf.apply(lambda row: deathToGank(row, "Kelody"), axis=1)

In [148]:
def countDeathsToGanks(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["deathToGank"] == True) & (kelodyEventsDf["victimName"] == name) & (kelodyEventsDf["timestamp"] <= 600000)])
kelody["deathsToGanks"] = kelody.apply(lambda row: countDeathsToGanks(row, "Kelody"), axis=1)

In [149]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks
6,EUW1_5808285447,7,1,0,1,0,0,0,0,1
11,EUW1_5808262634,2,2,2,2,1,2,0,1,0
26,EUW1_5803852070,7,1,11,0,1,3,0,1,1
36,EUW1_5803774910,7,1,7,1,0,2,0,1,0
41,EUW1_5803660385,2,2,5,1,0,3,0,0,0
51,EUW1_5803299160,2,1,2,0,6,1,0,2,1
61,EUW1_5803373057,2,3,1,1,1,2,1,1,0
71,EUW1_5803198068,2,1,3,1,1,3,0,0,0
86,EUW1_5803065394,7,1,3,5,2,4,1,1,0
96,EUW1_5802442397,7,3,7,2,1,1,0,1,1


In [150]:
participantsDf


,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5808285447,1,Kreatonn,TOP,Sion,0/7/6,16,7133,27598,644,...,4,12,1011,3181,0,3068,3047,3077,3340,True
1,EUW1_5808285447,2,Forcanist,JUNGLE,Darius,8/8/8,16,23119,42132,15433,...,6,11,3742,6631,4401,3111,3133,1031,3364,True
2,EUW1_5808285447,3,Hisshiden,MIDDLE,Ahri,24/5/5,18,35388,22712,9141,...,4,14,3020,6656,3157,3041,3135,3089,3364,True
3,EUW1_5808285447,4,Safiphy,BOTTOM,Aphelios,14/5/9,15,25949,17747,3910,...,3,4,2031,6671,6676,3031,3036,3006,3363,True
4,EUW1_5808285447,5,TarKy,UTILITY,Janna,3/4/24,15,7622,16187,15556,...,14,4,3853,2065,3107,3504,3158,3801,3364,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,EUW1_5747855273,6,Junipie,TOP,Illaoi,3/3/1,15,9241,10082,1507,...,4,12,6632,3047,3133,3181,1031,1037,3340,False
126,EUW1_5747855273,7,NightWõlf,JUNGLE,Viego,2/4/3,12,8777,19859,6826,...,4,11,3047,2031,6632,1053,1037,1042,3340,False
127,EUW1_5747855273,8,Kelody,MIDDLE,Ahri,0/9/1,10,8515,13651,1479,...,4,14,1056,6655,3020,1052,0,0,3340,False
128,EUW1_5747855273,9,Sir Nurum,BOTTOM,Xayah,2/9/1,10,8737,14138,865,...,7,4,1018,0,6671,3006,1037,3134,3340,False


In [151]:
def countWardsPlaced(row, name):
    participantId = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["participant"].values[0]
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["creatorId"] == participantId) & (kelodyEventsDf["type"] == "WARD_PLACED")])
kelody["wardsPlaced"] = kelody.apply(lambda row: countWardsPlaced(row, "Kelody"), axis=1)

In [152]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks,wardsPlaced
6,EUW1_5808285447,7,1,0,1,0,0,0,0,1,3
11,EUW1_5808262634,2,2,2,2,1,2,0,1,0,5
26,EUW1_5803852070,7,1,11,0,1,3,0,1,1,5
36,EUW1_5803774910,7,1,7,1,0,2,0,1,0,7
41,EUW1_5803660385,2,2,5,1,0,3,0,0,0,4
51,EUW1_5803299160,2,1,2,0,6,1,0,2,1,11
61,EUW1_5803373057,2,3,1,1,1,2,1,1,0,6
71,EUW1_5803198068,2,1,3,1,1,3,0,0,0,5
86,EUW1_5803065394,7,1,3,5,2,4,1,1,0,6
96,EUW1_5802442397,7,3,7,2,1,1,0,1,1,8


In [153]:
def countWardsPlacedBeforeGanked(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["deathToGank"] == True) & (kelodyEventsDf["victimName"] == name) & (kelodyEventsDf["timestamp"] <= 600000)])

    #TODO

In [158]:
def checkRoamKills(row, name):
    if row["assistingParticipantIds"] and (row["killerName"] == name) and (row["type"] == "CHAMPION_KILL") and (row["timestampMIN"] <= 15):
        return True
    return False

def countRoamKills(row):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & ((kelodyEventsDf["kelodyAssist"] == True) & (kelodyEventsDf["timestampMIN"] <= 15) | (kelodyEventsDf["kelodyRoamKill"] == True))])

In [159]:
kelodyEventsDf["kelodyRoamKill"] = kelodyEventsDf.apply(lambda row: checkRoamKills(row, "Kelody"), axis = 1)
kelody["roamKillsAssists"] = kelody.apply(lambda row: countRoamKills(row), axis=1)

In [173]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks,wardsPlaced,roamKillsAssists
6,EUW1_5808285447,7,1,0,1,0,0,0,0,1,3,6
11,EUW1_5808262634,2,2,2,2,1,2,0,1,0,5,8
26,EUW1_5803852070,7,1,11,0,1,3,0,1,1,5,6
36,EUW1_5803774910,7,1,7,1,0,2,0,1,0,7,5
41,EUW1_5803660385,2,2,5,1,0,3,0,0,0,4,9
51,EUW1_5803299160,2,1,2,0,6,1,0,2,1,11,9
61,EUW1_5803373057,2,3,1,1,1,2,1,1,0,6,7
71,EUW1_5803198068,2,1,3,1,1,3,0,0,0,5,7
86,EUW1_5803065394,7,1,3,5,2,4,1,1,0,6,2
96,EUW1_5802442397,7,3,7,2,1,1,0,1,1,8,8


In [ ]:
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
kelodyEventsDf.to_json('kelodyEvents.json', orient='records')
kelody.to_json('kelodyStats.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
